# 1. Install Dependencies

In [36]:
!pip install torch

In [37]:
!pip install transformers ipywidgets gradio --upgrade

In [38]:
!pip install sentencepiece

In [39]:
import gradio as gr                 # UI Library
from transformers import pipeline   # Transformers pipeline
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# 2. Load Up Pipeline

In [40]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [41]:
tokenizer.src_lang = "pl"
encoded_hi = tokenizer("Polska jest najlepsza", return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

Poland is the best


# 3. Create Gradio Function and Interface

In [58]:
# If u want to add language, just enter 'language_name': 'ISO 639-1 Code' pair
lang_dict = {'English': 'en', 'German': 'de', 'French': 'fr', 'Spanish': 'es', 'Polish': 'pl', 'Italian': 'it', 'Portugal': 'pt', 'Russian': 'ru', 'Chinese': 'zh'}

['English', 'German', 'French', 'Spanish', 'Polish', 'Italian', 'Portugal', 'Russian', 'Chinese']


In [80]:
def translate_transformers(from_text, choose_source_lang, choose_output_lang):
    for item, value in lang_dict.items():
        if choose_source_lang == item:
            source_lang = value
        if choose_output_lang == item:
            output_lang = value

    tokenizer.src_lang = source_lang
    encoded_hi = tokenizer(from_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id(output_lang))
    
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        

In [81]:
interface = gr.Interface(fn=translate_transformers,
    inputs = [gr.inputs.Textbox(lines=2, placeholder='Text to translate'), gr.inputs.Radio([item for item in lang_dict]), 
              gr.inputs.Radio([item for item in lang_dict])],
    outputs='text',
    title="Translator",
    description="Choose translation direction. Enjoy!"
)

In [82]:
interface.launch()
# interface.launch(share=True, auth=('patrinhoo', 'Patryk1996.'))

Running on local URL:  http://127.0.0.1:7879/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7879/', None)